In [1]:
from keras import applications
from keras.models import *
from keras.applications import *
from keras.preprocessing import *

dogs = [
 'n02085620','n02085782','n02085936','n02086079'
,'n02086240','n02086646','n02086910','n02087046'
,'n02087394','n02088094','n02088238','n02088364'
,'n02088466','n02088632','n02089078','n02089867'
,'n02089973','n02090379','n02090622','n02090721'
,'n02091032','n02091134','n02091244','n02091467'
,'n02091635','n02091831','n02092002','n02092339'
,'n02093256','n02093428','n02093647','n02093754'
,'n02093859','n02093991','n02094114','n02094258'
,'n02094433','n02095314','n02095570','n02095889'
,'n02096051','n02096177','n02096294','n02096437'
,'n02096585','n02097047','n02097130','n02097209'
,'n02097298','n02097474','n02097658','n02098105'
,'n02098286','n02098413','n02099267','n02099429'
,'n02099601','n02099712','n02099849','n02100236'
,'n02100583','n02100735','n02100877','n02101006'
,'n02101388','n02101556','n02102040','n02102177'
,'n02102318','n02102480','n02102973','n02104029'
,'n02104365','n02105056','n02105162','n02105251'
,'n02105412','n02105505','n02105641','n02105855'
,'n02106030','n02106166','n02106382','n02106550'
,'n02106662','n02107142','n02107312','n02107574'
,'n02107683','n02107908','n02108000','n02108089'
,'n02108422','n02108551','n02108915','n02109047'
,'n02109525','n02109961','n02110063','n02110185'
,'n02110341','n02110627','n02110806','n02110958'
,'n02111129','n02111277','n02111500','n02111889'
,'n02112018','n02112137','n02112350','n02112706'
,'n02113023','n02113186','n02113624','n02113712'
,'n02113799','n02113978']

cats=[
'n02123045','n02123159','n02123394','n02123597'
,'n02124075','n02125311','n02127052']


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


## 全局变量设置

In [2]:
pick_class='cat'   #需要预测的类型,猫和狗分开

#windows path
#out_path="data\\anormal"
#ori_path="data\\test_pick\\"+pick_class  

#linux path
out_path="data/anormal"
ori_path="data/test_pick/"+pick_class  

batch_size=125   #共12500张,batch_size=125,则100个bacth
top_n=50


## 预测方法

In [3]:
from keras.preprocessing.image import *
from keras.layers import *
import h5py
import shutil

def pick_not_same(MODEL, image_size, decode_func, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)   #预处理函数

    model = MODEL(input_tensor=x, weights='imagenet', include_top=True)  #含顶部全连接层
    
    #gen = ImageDataGenerator(rescale = 1./255) #实践证明,预测时不能做rescale
    gen = ImageDataGenerator()

    test_generator = gen.flow_from_directory(ori_path, image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)
    
    # 测试图像的数量
    image_numbers = test_generator.samples
    print ("test_generator images numbers={}".format(image_numbers))
    
    # 预测
    y_pred = model.predict_generator(test_generator, test_generator.samples//batch_size, verbose=1)
    print(y_pred.shape)
    y_pred_n = decode_func(y_pred, top=top_n)
    
    # 是否同一类别: 在模型预测top_n中有同样的类型，则认为是同一类别
    if pick_class == 'cat' :
        BAGS=cats   
    elif pick_class == 'dog':
        BAGS=dogs  
    
    result = np.zeros(len(y_pred_n))
    
    for i in range(len(y_pred_n)):   #第i张图片
        flag=False
        for j in range(len(y_pred_n[i])):   #top_n中的第j个结果 
            if y_pred_n[i][j][0] in BAGS:
                flag=True
                break
        if flag:
            result[i]=1
    
    filenames = test_generator.filenames

    for i in range(len(result)):
        if result[i]==0:
            print (filenames[i])
            #window path
            #out_file=out_path+'\\'+filenames[i]
            #ori_file=ori_path+'\\'+filenames[i]
            
            #linux path
            out_file=out_path+'/'+filenames[i]
            ori_file=ori_path+'/'+filenames[i]
            print(out_file)
            shutil.copy(ori_file, out_file)  #复制判断为不同类的图片
            
    print("{} pick up over".format(MODEL.__name__))
        
    return 

    

In [4]:
pick_not_same(InceptionV3, (299, 299), inception_v3.decode_predictions, inception_v3.preprocess_input)

pick_not_same(Xception, (299, 299), xception.decode_predictions, xception.preprocess_input)

pick_not_same(InceptionResNetV2, (299, 299), inception_resnet_v2.decode_predictions, inception_resnet_v2.preprocess_input)


Found 12500 images belonging to 1 classes.
test_generator images numbers=12500
100/100 [==============================] - 157s 2s/step
(12500, 1000)
cat/cat.10029.jpg
data/anormal/cat/cat.10029.jpg
cat/cat.10700.jpg
data/anormal/cat/cat.10700.jpg
cat/cat.10712.jpg
data/anormal/cat/cat.10712.jpg
cat/cat.10946.jpg
data/anormal/cat/cat.10946.jpg
cat/cat.11184.jpg
data/anormal/cat/cat.11184.jpg
cat/cat.11222.jpg
data/anormal/cat/cat.11222.jpg
cat/cat.11231.jpg
data/anormal/cat/cat.11231.jpg
cat/cat.11565.jpg
data/anormal/cat/cat.11565.jpg
cat/cat.11607.jpg
data/anormal/cat/cat.11607.jpg
cat/cat.11777.jpg
data/anormal/cat/cat.11777.jpg
cat/cat.12227.jpg
data/anormal/cat/cat.12227.jpg
cat/cat.12272.jpg
data/anormal/cat/cat.12272.jpg
cat/cat.12378.jpg
data/anormal/cat/cat.12378.jpg
cat/cat.1485.jpg
data/anormal/cat/cat.1485.jpg
cat/cat.2337.jpg
data/anormal/cat/cat.2337.jpg
cat/cat.2433.jpg
data/anormal/cat/cat.2433.jpg
cat/cat.2457.jpg
data/anormal/cat/cat.2457.jpg
cat/cat.2509.jpg
data/anor